In [1]:
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input,decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
import numpy as np
import cv2
import PIL
import tensorflow as tf
import os
from skimage import io, color


In [2]:
X= []
# print(os.listdir('C:/Users/HP/img/train'))
# print(os.listdir('/img/train'))
for file in os.listdir('C:/Users/HP/img/train'):
    
#     print(file)
    X.append(image.img_to_array(image.load_img('C:/Users/HP/img/train/' + file,target_size=(256,256))))
# print(X)
X =np.array(X, dtype=float)


split=int(0.95*len(X))
x_train = X[:split]*1/255

Xtest = X[split:]*1/255



[array([[[115.,  85.,  87.],
        [121.,  74.,  90.],
        [118.,  74.,  71.],
        ...,
        [136., 136., 134.],
        [138., 137., 135.],
        [137., 137., 135.]],

       [[117.,  77.,  65.],
        [101.,  68.,  63.],
        [116.,  84.,  61.],
        ...,
        [137., 137., 135.],
        [137., 137., 135.],
        [136., 136., 136.]],

       [[124.,  91.,  82.],
        [119.,  93.,  80.],
        [109.,  82.,  73.],
        ...,
        [136., 136., 134.],
        [136., 138., 137.],
        [137., 137., 137.]],

       ...,

       [[120.,  92.,  55.],
        [ 85.,  47.,  36.],
        [111.,  81.,  47.],
        ...,
        [ 30.,  16.,  13.],
        [ 27.,  16.,  20.],
        [ 16.,   6.,  14.]],

       [[119.,  90.,  74.],
        [118.,  95.,  54.],
        [ 99.,  62.,  56.],
        ...,
        [ 47.,  22.,  17.],
        [ 45.,  19.,  18.],
        [ 39.,  20.,  16.]],

       [[ 93.,  62.,  44.],
        [ 81.,  47.,  45.],
        [110., 

In [3]:
#Model Definition
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))



model.compile(optimizer='rmsprop', loss='mse')

In [4]:
# Image Augmentation by ImageDataGenerator
datagen = image.ImageDataGenerator(shear_range=0.2,
                             zoom_range=0.3,
                             horizontal_flip=True,
                            rotation_range=20,
                            vertical_flip=True)
batch_size=20
def get_from_gen(batch_size):
    for batch in datagen.flow(x_train,batch_size=batch_size):
        lab_batch = color.rgb2lab(batch)
        x_batch = lab_batch[:,:,:,0]
        y_batch = lab_batch[:,:,:,1:]/128
        yield(x_batch.reshape(x_batch.shape + (1,)),y_batch)
        
        

In [ ]:
#train
model.fit_generator(get_from_gen(batch_size),steps_per_epoch=10,epochs=1)

#test
x_test = color.rgb2lab(Xtest)[:,:,:,0]
x_test = Xtest.reshape(Xtest.shape + (1,))
y_test = color.rgb2lab(Xtest)[:,:,:,1:]
y_test = y_test / 128

print(model.evaluate(x_test, y_test,batch_size=batch_size))

Epoch 1/1
